# Jupyter with PostgreSQL

## Libararies and Settings

In [ ]:
# Libraries
import os
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(os.getcwd())

## Create database connection

In [ ]:
conn = psycopg2.connect("host=localhost dbname=app user=app_user password=app_password")

## Read data

In [ ]:
df = pd.read_csv('/workspace/apartments_data_prepared.csv', sep=',')
df.head(5)

## Write data to database

In [ ]:
engine = create_engine('postgresql://app_user:app_password@db:5432/postgres')
df.to_sql('apartment_table', engine, if_exists='replace')

## Make SQL query

In [ ]:
df_sub = pd.read_sql_query('''SELECT
                             address_raw,
                             rooms,
                             area,
                             price
                             FROM apartment_table''', 
                          con=engine)
df_sub.head()

## Plot apartment prices

In [ ]:
# Plot Histogram
fig = plt.figure( figsize=(7,4))
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
n, bins, patches = plt.hist(x=df['price'], 
                            bins=20, 
                            color='#5DADE2',
                            alpha=1.00, 
                            rwidth=0.95
                   )
plt.grid(True)
plt.ticklabel_format(style='plain')
plt.grid(axis='y', alpha=0.75)

# Set labels
plt.xlabel('price', fontsize=10, labelpad=10)
plt.ylabel('Frequency', fontsize=10, labelpad=10)
plt.title('Histogram of apartment prices', fontsize=12, pad=10)

# Set fontsize of tick labels
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.show()

## Check whether the Postgis extention for PostgreSQL is installed

In [ ]:
# Function
def check_postgis_installed():
    # Database connection parameters
    db_params = {
        'dbname': 'app',
        'user': 'app_user',
        'password': 'app_password',
        'host': 'localhost',
        'port': 5432
    }

    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        # Query to check if PostGIS is installed
        cursor.execute("SELECT postgis_full_version();")
        postgis_version = cursor.fetchone()

        if postgis_version:
            print(f"PostGIS is installed: {postgis_version[0]}")
        else:
            print("PostGIS is not installed.")

        # Close the cursor and connection
        cursor.close()
        conn.close()
    except psycopg2.Error as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    check_postgis_installed()


## Check whether osm2pgsql ist installed

In [ ]:
import subprocess

def check_osm2pgsql():
    try:
        # Run the osm2pgsql version command
        result = subprocess.run(['osm2pgsql', '--version'], capture_output=True, text=True)
        
        # Check if the command was successful
        if result.returncode == 0:
            print("osm2pgsql is installed and working correctly.")
            print(result.stdout)
        else:
            print("osm2pgsql is not working correctly.")
            print(result.stderr)
    
    except FileNotFoundError:
        print("osm2pgsql is not installed.")

if __name__ == "__main__":
    check_osm2pgsql()

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')